In [1]:
import os
# os.chdir('../')
import pandas as pd
import numpy as np
from DeepMTP.dataset import load_process_MLC, load_process_MTR, load_process_DP, process_dummy_MLC, process_dummy_MTR, process_dummy_DP, load_process_MC, load_process_MTL
from DeepMTP.utils.data_utils import data_process, BaseDataset
from DeepMTP.utils.tests import check_mlc_results, check_mtr_results, check_dp_results, check_mtl_results
from DeepMTP.main import DeepMTP
from DeepMTP.utils.utils import generate_config
from tqdm import tqdm
from PIL import Image

/home/dimitriosi/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Matrix Completion
<img src="../images/mc_example.png"/>

# load-process-split

In [2]:
# load dataset
data = load_process_MC(dataset_name='ml-100k')
# process and split
train, val, test, data_info = data_process(data, validation_setting='A', verbose=True)
# sanity check
check_mc_results(train, val, test)

Interaction file: triplet format detected
Interaction file: checking format consistency... Passed
Interaction file: checking instance id format consistency... Passed
Interaction file: checking target id type consistency... Passed

Interaction file: checking target variable type consistency... Passed
Automatically detected type of target variable type: real-valued

-- Test set was not provided, could not detect if novel instances exist or not 
-- Test set was not provided, could not detect if novel targets exist or not 



Splitting train to train-test according to validation setting A... Done
Splitting train to train-val according to validation setting A... Done
Checking if MC splitting results are valid... 

In [3]:
train

{'y': {'data':        instance_id  target_id  value
  0                0          0      3
  1                1          1      3
  2                2          2      1
  3                3          3      2
  4                4          4      1
  ...            ...        ...    ...
  99994          369        313      3
  99995          875        173      3
  99996          708        247      5
  99997           37       1004      1
  99998           58        443      2
  
  [72000 rows x 3 columns],
  'original_format': 'triplets',
  'instance_id_type': 'int',
  'target_id_type': 'int',
  'missing_values': True},
 'X_instance': None,
 'X_target': None}

In [4]:
val

{'y': {'data':        instance_id  target_id  value
  58866          718       1076      4
  55412          379        249      3
  93518          434        347      3
  88720          739       1079      4
  58369          794        357      5
  ...            ...        ...    ...
  23233          234        157      4
  55365          579        172      3
  24702          409       1028      4
  43678          498        251      2
  65247          335        229      5
  
  [8000 rows x 3 columns]},
 'X_instance': None,
 'X_target': None}

In [5]:
test

{'y': {'data':        instance_id  target_id  value
  75721          873        377      4
  80184          808        601      3
  19864           90        354      4
  76699          409        570      2
  92991          496        356      2
  ...            ...        ...    ...
  32595           53        204      5
  29313          511        528      5
  37862          602        487      4
  53421          646        300      4
  42410           15        549      3
  
  [20000 rows x 3 columns]},
 'X_instance': None,
 'X_target': None}

In [6]:
data_info

{'detected_validation_setting': 'A',
 'detected_problem_mode': 'regression',
 'instance_branch_input_dim': 943,
 'target_branch_input_dim': 1682}

# Configure and train network

In [7]:
config = generate_config(    
    instance_branch_input_dim = data_info['instance_branch_input_dim'],
    target_branch_input_dim = data_info['target_branch_input_dim'],
    validation_setting = data_info['detected_validation_setting'],
    general_architecture_version = 'dot_product',
    problem_mode = data_info['detected_problem_mode'],
    learning_rate = 0.001,
    decay = 0,
    batch_norm = False,
    dropout_rate = 0,
    momentum = 0.9,
    weighted_loss = False,
    compute_mode = 'cuda:1',
    train_batchsize = 1024,
    val_batchsize = 1024,
    num_epochs = 50,
    num_workers = 8,
    metrics = ['RMSE', 'MSE'],
    metrics_average = ['micro'],
    patience = 10,

    evaluate_train = True,
    evaluate_val = True,

    verbose = True,
    results_verbose = False,
    use_early_stopping = True,
    use_tensorboard_logger = True,
    wandb_project_name = 'DeepMTP_v2',
    wandb_project_entity = 'diliadis',
    metric_to_optimize_early_stopping = 'loss',
    metric_to_optimize_best_epoch_selection = 'loss',

    instance_branch_architecture = 'MLP',
    use_instance_features = False,
    instance_branch_params = {
        'instance_branch_nodes_reducing_factor': 2,
        'instance_branch_nodes_per_layer': [512, 256],
        'instance_branch_layers': None,
        # 'instance_branch_conv_architecture': 'resnet',
        # 'instance_branch_conv_architecture_version': 'resnet101',
        # 'instance_branch_conv_architecture_dense_layers': 1,
        # 'instance_branch_conv_architecture_last_layer_trained': 'last',
    },

    target_branch_architecture = 'MLP',
    use_target_features = False,
    target_branch_params = {
        'target_branch_nodes_reducing_factor': 2,
        'target_branch_nodes_per_layer': [512, 256],
        'target_branch_layers': None,
        # 'target_branch_conv_architecture': 'resnet',
        # 'target_branch_conv_architecture_version': 'resnet101',
        # 'target_branch_conv_architecture_dense_layers': 1,
        # 'target_branch_conv_architecture_last_layer_trained': 'last',
    },

    embedding_size = 100,
    comb_mlp_nodes_reducing_factor = 2,
    comb_mlp_nodes_per_layer = [2048, 2048, 2048],
    comb_mlp_layers = None, 

    save_model = True,

    eval_every_n_epochs = 10,

    additional_info = {})

In [8]:
# initialize model
model = DeepMTP(config)
print(str(model.deepMTP_model))
# train, validate, test
validation_results = model.train(train, val, test)

Selected device: cuda:1


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


TwoBranchDotProductModel(
  (instance_branch_model): MLP(
    (predictor): ModuleList(
      (0): Linear(in_features=943, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=256, out_features=100, bias=True)
      (5): LeakyReLU(negative_slope=0.01)
    )
  )
  (target_branch_model): MLP(
    (predictor): ModuleList(
      (0): Linear(in_features=1682, out_features=512, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=256, out_features=100, bias=True)
      (5): LeakyReLU(negative_slope=0.01)
    )
  )
)
Starting training...


wandb: Currently logged in as: diliadis. Use `wandb login --relogin` to force relogin


Epoch:0... Done
  Validating... Calculating val performance... Done
Done
Epoch:1... Done
  Validating... Done
Epoch:2... Done
  Validating... Done
Epoch:3... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 2
Epoch:4... Done
  Validating... Done
Epoch:5... Done
  Validating... Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 4
Epoch:6... Done
  Validating... Done
Epoch:7... Done
  Validating... Done
Epoch:8... Done
  Validating... Done
Epoch:9... Done
  Validating... Done
Epoch:10... Done
  Validating... Calculating val performance... Done
Done
-----------------------------EarlyStopping counter: 1 out of 10---------------------- best epoch currently 9
Epoch:11... Done
  Validating... Done
-----------------------------EarlyStopping counter: 2 out of 10---------------------- best epoch currently 9
Epoch:12... Done
  Validating... Done
--------

test_MSE_micro,▁
test_RMSE_micro,▁
train_MSE_micro,█▁▁▁▁
train_RMSE_micro,█▂▁▁▁
train_loss,█▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_MSE_micro,█▇▃▁▁
val_RMSE_micro,█▇▃▁▁
val_loss,█▇▇▇▆▆▆▅▅▅▇▅▆▂▃▆▂▆▂▅▂▂▂▂▂▁▂▃▃▂▁▁▁▁▁▂▂▂▃▄
test_MSE_micro,0.85876
test_RMSE_micro,0.92669
train_MSE_micro,0.71136


+-------+--------+--------+------------+-----------+
|  mode | #epoch |  loss  | RMSE_micro | MSE_micro |
+-------+--------+--------+------------+-----------+
| train |   0    | 2.943  |   1.7209   |   2.9616  |
| train |   1    | 0.9021 |     -      |     -     |
| train |   2    | 0.896  |     -      |     -     |
| train |   3    | 0.8884 |     -      |     -     |
| train |   4    | 0.8829 |     -      |     -     |
| train |   5    | 0.8801 |     -      |     -     |
| train |   6    | 0.8762 |     -      |     -     |
| train |   7    | 0.869  |     -      |     -     |
| train |   8    | 0.8698 |     -      |     -     |
| train |   9    | 0.8656 |     -      |     -     |
| train |   10   | 0.8646 |   0.9297   |   0.8644  |
| train |   11   | 0.8609 |     -      |     -     |
| train |   12   | 0.8546 |     -      |     -     |
| train |   13   | 0.837  |     -      |     -     |
| train |   14   | 0.8213 |     -      |     -     |
| train |   15   | 0.8072 |     -      |     -

# Inference

In [ ]:
# generate predictions from the trained model
results, preds = model.predict(train, return_predictions=True ,verbose=True)

# Continue training model

In [ ]:
# this is a minimal configuration needed for HPO methods like Hyperband
config = {
    'verbose': True,
    'num_epochs': 20,
    'num_workers': 8,
    # 'wandb_project_name': None,
    # 'wandb_project_entity': None,
    # 'use_tensorboard_logger': False
}
# initialize the model and load the pretrained weights etc.
new_model = DeepMTP(config, 'add path to model.pt')

new_model.train(train, val, test)